In [41]:
from imutils.video import VideoStream
import argparse
import imutils
import time
import cv2
import sys
import numpy as np
import os

In [32]:
import cv2
import numpy as np

all_camera_idx_available = []

for camera_idx in range(-100, 900):
    cap = cv2.VideoCapture(camera_idx)
    if cap.isOpened():
        print(f'Camera index available: {camera_idx}')
        all_camera_idx_available.append(camera_idx)
        cap.release()

Camera index available: 0


In [25]:
vs.stop()

In [45]:
arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
arucoParams = cv2.aruco.DetectorParameters_create()
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

[INFO] starting video stream...


In [46]:
while True:
    frame = vs.read()
    if frame == None:
        time.sleep(1.0)
        print("[INFO] no valid input...")
        continue
    image = imutils.resize(frame, width = 1000)
    (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict, parameters=arucoParams)
    if len(corners) > 0:
        # flatten the ArUco IDs list
        ids = ids.flatten()
        # loop over the detected ArUCo corners
        for (markerCorner, markerID) in zip(corners, ids):
            # extract the marker corners (which are always returned in
            # top-left, top-right, bottom-right, and bottom-left order)
            #print(markerCorner)
            print(markerCorner)
            #corners = markerCorner.reshape((4, 1))
            (topLeft, topRight, bottomRight, bottomLeft) = markerCorner[0]
            # convert each of the (x, y)-coordinate pairs to integers
            topRight = (int(topRight[0]), int(topRight[1]))
            bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
            bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
            topLeft = (int(topLeft[0]), int(topLeft[1]))

            # draw the bounding box of the ArUCo detection
            cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
            cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
            cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
            cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)
            # compute and draw the center (x, y)-coordinates of the ArUco
            # marker
            cX = int((topLeft[0] + bottomRight[0]) / 2.0)
            cY = int((topLeft[1] + bottomRight[1]) / 2.0)
            cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)
            # draw the ArUco marker ID on the image
            cv2.putText(image, str(markerID),
                (topLeft[0], topLeft[1] - 15), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (0, 255, 0), 2)
            print("[INFO] ArUco marker ID: {}".format(markerID))
    # show the output image
    cv2.imshow("Image", image)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break
cv2.destroyAllWindows()
vs.stop()

[INFO] no valid input...
[INFO] no valid input...
[INFO] no valid input...
[INFO] no valid input...
[INFO] no valid input...
[INFO] no valid input...
[INFO] no valid input...
[INFO] no valid input...
[INFO] no valid input...
[INFO] no valid input...
[INFO] no valid input...
[INFO] no valid input...


KeyboardInterrupt: 